In [1]:
from batch_gen import BatchGenerator
from model import MultiStageModel
from random import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

In [2]:
vid_list_file = 'C:/github/casual_tcn/cholec80/train.txt'
mapping_file = 'C:/github/casual_tcn/cholec80/action_mapping.txt'
feature_dir = 'C:/github/casual_tcn/cholec80/train_dataset/video_feature@2020/'
gt_dir = 'C:/github/casual_tcn/cholec80/train_dataset/annotation_folder/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_stages = 4      # number of SingleStageModel in MultiStageModel.stages
num_layers = 10     # number of DilatedResidualLayer in SingleStageModel.layers
num_f_maps = 64     # number of kernels in DilatedResidualLayer and conv1d layer
features_dim = 2048 # each feature vector contains 2048 floats
bz = 1              # batch size
lr = 0.0005         # learning rate
num_epochs = 50     # training epoches


In [3]:
features = np.load(feature_dir + 'video01.npy').T
input_x = torch.tensor(features, dtype=torch.float).unsqueeze(0)
input_x = input_x.to(device)
model = MultiStageModel(num_stages, num_layers, num_f_maps, features_dim, num_classes=6)
model.to(device)
predictions = model(input_x, torch.ones(input_x.size(), device=device))

In [7]:
print(predictions.shape)
print(predictions[-1].data.shape)
print(predictions[-1].shape)

torch.Size([5, 1, 6, 8666])
torch.Size([1, 6, 8666])
torch.Size([1, 6, 8666])


In [22]:
something, predicted = torch.max(predictions[-1].data, 1)
print(predictions[-1].data.shape)
print(predicted.shape)
print(predicted)
predicted = predicted.squeeze()
print(predicted.shape)
print(predicted)
print(predicted[0].item())

torch.Size([1, 6, 8666])
torch.Size([1, 8666])
tensor([[0, 0, 4,  ..., 0, 3, 0]], device='cuda:0')
torch.Size([8666])
tensor([0, 0, 4,  ..., 0, 3, 0], device='cuda:0')
0


In [14]:
file_ptr = open(mapping_file, 'r')
actions = file_ptr.read().split('\n')[:-1] # extract each line into array 'actions'
file_ptr.close()
actions_dict = dict()
for a in actions:
    actions_dict[a.split()[1]] = int(a.split()[0]) # construct a dictionary: operation_name -> index

In [42]:
def read_file(path):
    with open(path, 'r') as f:
        content = f.read()
        f.close()
    return content

gt_dir = "C:/github/casual_tcn/cholec80/train_dataset/annotation_folder/"
recog_path = "./results/"
file_list = "C:/github/casual_tcn/cholec80/test.txt"

gt_content = read_file(gt_dir+"video01.txt").split('\n')[0:-1]
for i in range(len(gt_content)):
    gt_content[i] = gt_content[i].split('\t')[1]

In [45]:
recog_file = recog_path + "video06.txt"
recog_content = read_file(recog_file).split('\n')[1].split()

In [50]:
import os
train_vid = [1,20]
test_vid = [21,40]
train_list_file = "C:/github/casual_tcn/cholec80/train.txt"
test_list_file = "C:/github/casual_tcn/cholec80/test.txt"

if os.path.exists(train_list_file):
    os.remove(train_list_file)
file_ptr = open(train_list_file, "w")
for i in range(train_vid[0], train_vid[1]+1):
    file_ptr.write("video%02d.mp4\n" % (i))
file_ptr.close()

if os.path.exists(test_list_file):
    os.remove(test_list_file)
file_ptr = open(test_list_file, "w")
for i in range(test_vid[0], test_vid[1]+1):
    file_ptr.write("video%02d.mp4\n" % (i))
file_ptr.close()
